In [1]:
import json
import pandas as pd
import numpy as np
import os

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm

from scipy.stats import kendalltau, spearmanr

from rmatrix.classification import RMatrixClassifier
from decision_rules.serialization.utils import JSONSerializer

from decision_rules.classification.ruleset import ClassificationRuleSet
from decision_rules.measures import c2, precision

In [2]:
def mutual_inclusion(R, M):
  """
  This function takes two lists, R and M, and returns the result of the equation:

  F(R) n F(M) = F(R) U F(M)

  where n is the intersection and U is the union of the sets.

  Args:
    R: A list of elements.
    M: A list of elements.

  Returns:
    A list containing the elements in the intersection and union of R and M.
  """

  # Find the intersection of R and M using a set
  intersection = set(R).intersection(set(M))

  # Find the union of R and M using the + operator
  union = set(R).union(set(M))

  # Combine the intersection and union into a single list
  result = len(intersection)/len(union) 

  return result


def rmatrix_unique_rules(rule_str, feature_names=None):
    features = []
    conditions = rule_str.split(" AND ")
    for condition in conditions:
        feature_value = condition.split(" = ")
        features.append(feature_value[0])
    
    return list(pd.unique(features))

def anchor_unique_rules(rule_str, feature_names=None):
    features = []
    conditions = rule_str.split(" AND ")
    for condition in conditions:
      for feature in feature_names:
        if feature in condition:
          features.append(feature)
    
    return list(pd.unique(features))

def extract_shap_fi(shap_df_row, shap_df, shap_length, stat):
    row_values = np.abs(shap_df.iloc[shap_df_row,:].values)
    col_names = np.array(shap_df.columns)
    row_values_sort_idx = np.argsort(-row_values)
    if stat == "all":
      feature_top = list(col_names[row_values_sort_idx])
    elif stat == "max":
      top = np.max(shap_length)
      feature_top = list(col_names[row_values_sort_idx])[:top]
    elif stat == "len":
      top = shap_length[shap_df_row]
      feature_top = list(col_names[row_values_sort_idx])[:top]
    return feature_top

def calculate_correlations(fi_rule, fi_bb):
    rank_bb = fi_bb.loc[fi_rule]["bb_rank"].values

    tau = kendalltau(list(range(1,len(fi_rule)+1)), rank_bb)
    spearman = spearmanr(list(range(1,len(fi_rule)+1)), rank_bb)
    return 0 if np.isnan(tau[0]) else tau[0], 0 if np.isnan(spearman[0]) else spearman[0]

def precision(c) -> float:  # pylint: disable=missing-function-docstring
    if (c.p + c.n) == 0:
        return 0
    return c.p / (c.p + c.n)

def coverage(c) -> float:  # pylint: disable=missing-function-docstring
    return c.p / c.P

def shap_rank(shap_df, shap_df_row):
    row_values = np.abs(shap_df.iloc[shap_df_row,:].values)
    col_names = np.array(shap_df.columns)
    shap_bb = pd.DataFrame({'attribute': col_names, 'importance': row_values})

    shap_bb = shap_bb.sort_values(by="importance", ascending=False).set_index("attribute")
    shap_bb["bb_rank"] = shap_bb["importance"].rank(method="min", ascending=False)

    return shap_bb

In [3]:
bb_models = pd.read_csv("../results/selected_bb_models.csv")
datasets = bb_models["dataset"].unique()

Global

In [6]:
class_types = ["_filterFF_c2_global", "_filterFF_precision_global"]

results_all = pd.DataFrame()

for class_type in class_types:

    for dataset_path in tqdm(datasets):

        models = np.unique(bb_models[bb_models["dataset"] == dataset_path]["model"])

        x_train_df = pd.read_csv(f"../results_all/{dataset_path}/train.csv")
        x_train_df = x_train_df.rename(columns={'target': 'name'})
        y_train = x_train_df["name"].squeeze().astype(str)
        x_train_df.drop(columns=["name"], inplace=True)

        x_test_df = pd.read_csv(f"../results_all/{dataset_path}/test.csv")
        x_test_df = x_test_df.rename(columns={'target': 'name'})
        y_test = x_test_df["name"].squeeze().astype(str)
        x_test_df.drop(columns=["name"], inplace=True)

        binary_columns = list(x_train_df.columns[x_train_df.isin([0,1]).all()])
        if len(binary_columns) > 0:
            x_train_df[binary_columns] = x_train_df[binary_columns].astype(str)
            x_test_df[binary_columns] = x_test_df[binary_columns].astype(str)

        feature_names = x_train_df.columns

        for model_path in models:

            fi_path = f"../results_new/{dataset_path}/{model_path}/fi_test.csv"
            file_path= f"../results_all/{dataset_path}/{model_path}/ruleset{class_type}.json"
            anchor_path = f"../results_anchor_all/{dataset_path}/{model_path}/anchor_train.csv"
            
            if os.path.exists(fi_path) and os.path.exists(file_path) and os.path.exists(anchor_path):

                anchor_train = pd.read_csv(anchor_path, sep=";")
                anchor_test = pd.read_csv(f"../results_anchor_all/{dataset_path}/{model_path}/anchor_test.csv", sep=";")

                fi = pd.read_csv(fi_path)
                fi = fi.sort_values("importance", ascending=False).set_index("attribute")
                fi["bb_rank"] = fi["importance"].rank(method="min", ascending=False)
                features = list(fi.index)
                
                with open(file_path, 'r') as json_file:
                    ruleset_json_read = json.load(json_file)

                classifier = JSONSerializer.deserialize(ruleset_json_read, target_class=ClassificationRuleSet)
                if "c2" in class_type:
                    classifier.update(x_train_df, y_train, measure=c2)
                elif "precision" in class_type:
                    classifier.update(x_train_df, y_train, measure=precision)

                rmatrix_rules = [rule.premise.to_string(feature_names) for rule in classifier.rules]
                rmatrix_precision = [precision(rule.coverage) for rule in classifier.rules]
                rmatrix_coverage = [coverage(rule.coverage) for rule in classifier.rules]
                anchor_rules = anchor_train["anchor_rule"].values

                mi_rmatrix_anchor = [mutual_inclusion(rmatrix_unique_rules(rmatrix_rules[obs], features), anchor_unique_rules(anchor_rules[obs], features)) for obs in range(len(rmatrix_rules))]

                rmatrix_length = [len(rmatrix_unique_rules(rule, features)) for rule in rmatrix_rules]
                mi_rmatrix_fi_all = [mutual_inclusion(rmatrix_unique_rules(rmatrix_rules[obs], features), features) for obs in range(len(rmatrix_rules))]
                mi_rmatrix_fi_len = [mutual_inclusion(rmatrix_unique_rules(rmatrix_rules[obs], features), features[:rmatrix_length[obs]]) for obs in range(len(rmatrix_rules))]

                anchor_length = [len(anchor_unique_rules(rule, features)) for rule in anchor_rules]
                mi_anchor_fi_all = [mutual_inclusion(anchor_unique_rules(anchor_rules[obs], features), features) for obs in range(len(anchor_rules))]
                mi_anchor_fi_len = [mutual_inclusion(anchor_unique_rules(anchor_rules[obs], features), features[:anchor_length[obs]]) for obs in range(len(anchor_rules))]

                corr_rmatrix = [calculate_correlations(rmatrix_unique_rules(rmatrix_rules[obs], features), fi) for obs in range(len(rmatrix_rules))]                
                corr_tau_rmatrix = [cor[0] for cor in corr_rmatrix]
                corr_sro_rmatrix = [cor[1] for cor in corr_rmatrix]

                corr_anchor = [calculate_correlations(anchor_unique_rules(anchor_rules[obs], features), fi) for obs in range(len(anchor_rules))]                    
                corr_tau_anchor = [cor[0] for cor in corr_anchor]
                corr_sro_anchor = [cor[1] for cor in corr_anchor]

                results = pd.DataFrame({'mi_rmatrix_anchor': mi_rmatrix_anchor, 
                                        'mi_rmatrix_fi_all': mi_rmatrix_fi_all, 'mi_rmatrix_fi_len': mi_rmatrix_fi_len,
                                        'mi_anchor_fi_all': mi_anchor_fi_all, 'mi_anchor_fi_len': mi_anchor_fi_len,
                                        'corr_tau_rmatrix': corr_tau_rmatrix, 'corr_tau_anchor': corr_tau_anchor,
                                        'rmatrix_precision': rmatrix_precision, 'rmatrix_coverage': rmatrix_coverage,
                                        'anchor_precision': anchor_train["anchor_precision_correct"].values, 
                                        'anchor_coverage': anchor_train["anchor_coverage_correct"].values
                                    })
                
                results["dataset"] = dataset_path
                results["model"] = model_path
                results["measure"] = class_type.split("_")[2]
                results_all = pd.concat([results_all, results])

            else:

                print(f"Problem with {dataset_path} {model_path}")

 13%|█▎        | 4/30 [00:11<01:16,  2.92s/it]

Problem with soybean RF_1
Problem with soybean SVC_0
Problem with soybean XGB_0


 43%|████▎     | 13/30 [00:40<00:52,  3.10s/it]

Problem with semeion RF_0
Problem with semeion SVC_0
Problem with semeion XGB_1


 63%|██████▎   | 19/30 [01:21<00:52,  4.78s/it]

...

Problem with mushroom RF_1


 93%|█████████▎| 28/30 [03:48<00:36, 18.43s/it]

Problem with har RF_2
Problem with har SVC_1
Problem with har XGB_1


100%|██████████| 30/30 [07:03<00:00, 14.11s/it]

Problem with nomao RF_2
Problem with nomao SVC_0
Problem with nomao XGB_2


In [7]:
len(np.unique(results_all["dataset"]))

21

In [8]:
results_all.to_csv(f"../results_anchor_all/anchor_mi_corr_all_global.csv", index=False)

Local

In [6]:
class_types = ["_filterFF_c2_local", "_filterFF_precision_local"]

results_all = pd.DataFrame()

for class_type in class_types:

    for dataset_path in tqdm(datasets):

        models = np.unique(bb_models[bb_models["dataset"] == dataset_path]["model"])

        x_train_df = pd.read_csv(f"../results_all/{dataset_path}/train.csv")
        x_train_df = x_train_df.rename(columns={'target': 'name'})
        y_train = x_train_df["name"].squeeze().astype(str)
        x_train_df.drop(columns=["name"], inplace=True)

        x_test_df = pd.read_csv(f"../results_all/{dataset_path}/test.csv")
        x_test_df = x_test_df.rename(columns={'target': 'name'})
        y_test = x_test_df["name"].squeeze().astype(str)
        x_test_df.drop(columns=["name"], inplace=True)

        binary_columns = list(x_train_df.columns[x_train_df.isin([0,1]).all()])
        if len(binary_columns) > 0:
            x_train_df[binary_columns] = x_train_df[binary_columns].astype(str)
            x_test_df[binary_columns] = x_test_df[binary_columns].astype(str)

        feature_names = x_train_df.columns

        for model_path in models:

            shap_path = f"../results_all/{dataset_path}/{model_path}/shap.csv"
            file_path= f"../results_all/{dataset_path}/{model_path}/ruleset{class_type}.json"
            anchor_path = f"../results_anchor_all/{dataset_path}/{model_path}/anchor_train.csv"
            
            if os.path.exists(shap_path) and os.path.exists(file_path) and os.path.exists(anchor_path):

                anchor_train = pd.read_csv(anchor_path, sep=";")
                anchor_test = pd.read_csv(f"../results_anchor_all/{dataset_path}/{model_path}/anchor_test.csv", sep=";")

                shap = pd.read_csv(shap_path)
                
                with open(file_path, 'r') as json_file:
                    ruleset_json_read = json.load(json_file)

                classifier = JSONSerializer.deserialize(ruleset_json_read, target_class=ClassificationRuleSet)
                if "c2" in class_type:
                    classifier.update(x_train_df, y_train, measure=c2)
                elif "precision" in class_type:
                    classifier.update(x_train_df, y_train, measure=precision)

                rmatrix_rules = [rule.premise.to_string(feature_names) for rule in classifier.rules]
                rmatrix_precision = [precision(rule.coverage) for rule in classifier.rules]
                rmatrix_coverage = [coverage(rule.coverage) for rule in classifier.rules]
                anchor_rules = anchor_train["anchor_rule"].values

                rules_features = [rmatrix_unique_rules(rule, feature_names) for rule in rmatrix_rules]
                rules_lengths = [len(rule) for rule in rules_features]

                shap_features = [extract_shap_fi(row, shap, rules_lengths, stat = "all") for row in range(len(shap))]                  
                shap_features_max = [extract_shap_fi(row, shap, rules_lengths, stat = "max") for row in range(len(shap))]
                shap_features_len = [extract_shap_fi(row, shap, rules_lengths, stat = "len") for row in range(len(shap))]

                mi_rmatrix_anchor = [mutual_inclusion(rmatrix_unique_rules(rmatrix_rules[obs], feature_names), anchor_unique_rules(anchor_rules[obs], feature_names)) for obs in range(len(rmatrix_rules))]

                rmatrix_length = [len(rmatrix_unique_rules(rule, feature_names)) for rule in rmatrix_rules]
                mi_rmatrix_fi_all = [mutual_inclusion(rmatrix_unique_rules(rmatrix_rules[obs], feature_names), shap_features[obs]) for obs in range(len(rmatrix_rules))]
                mi_rmatrix_fi_len = [mutual_inclusion(rmatrix_unique_rules(rmatrix_rules[obs], feature_names), shap_features_len[obs]) for obs in range(len(rmatrix_rules))]

                anchor_length = [len(anchor_unique_rules(rule, feature_names)) for rule in anchor_rules]
                mi_anchor_fi_all = [mutual_inclusion(anchor_unique_rules(anchor_rules[obs], feature_names), shap_features[obs]) for obs in range(len(anchor_rules))]
                mi_anchor_fi_len = [mutual_inclusion(anchor_unique_rules(anchor_rules[obs], feature_names), shap_features_len[obs]) for obs in range(len(anchor_rules))]

                corr_rmatrix = [calculate_correlations(rmatrix_unique_rules(rmatrix_rules[obs], feature_names), shap_rank(shap, obs)) for obs in range(len(rmatrix_rules))]                
                corr_tau_rmatrix = [cor[0] for cor in corr_rmatrix]
                corr_sro_rmatrix = [cor[1] for cor in corr_rmatrix]

                corr_anchor = [calculate_correlations(anchor_unique_rules(anchor_rules[obs], feature_names), shap_rank(shap, obs)) for obs in range(len(anchor_rules))]                    
                corr_tau_anchor = [cor[0] for cor in corr_anchor]
                corr_sro_anchor = [cor[1] for cor in corr_anchor]

                results = pd.DataFrame({'mi_rmatrix_anchor': mi_rmatrix_anchor, 
                                        'mi_rmatrix_fi_all': mi_rmatrix_fi_all, 'mi_rmatrix_fi_len': mi_rmatrix_fi_len,
                                        'mi_anchor_fi_all': mi_anchor_fi_all, 'mi_anchor_fi_len': mi_anchor_fi_len,
                                        'corr_tau_rmatrix': corr_tau_rmatrix, 'corr_tau_anchor': corr_tau_anchor,
                                        'rmatrix_precision': rmatrix_precision, 'rmatrix_coverage': rmatrix_coverage,
                                        'anchor_precision': anchor_train["anchor_precision_correct"].values, 
                                        'anchor_coverage': anchor_train["anchor_coverage_correct"].values
                                    })
                
                results["dataset"] = dataset_path
                results["model"] = model_path
                results["measure"] = class_type.split("_")[2]
                results_all = pd.concat([results_all, results])

            else:

                print(f"Problem with {dataset_path} {model_path}")

  0%|          | 0/30 [00:00<?, ?it/s]

Problem with cylinder-bands SVC_1


  3%|▎         | 1/30 [00:03<01:48,  3.74s/it]

Problem with wdbc SVC_1


  7%|▋         | 2/30 [00:07<01:41,  3.62s/it]

...

Problem with har RF_2
Problem with har SVC_1
Problem with har XGB_1


 97%|█████████▋| 29/30 [06:24<00:41, 41.33s/it]

Problem with nursery SVC_0
Problem with nursery XGB_1


100%|██████████| 30/30 [06:24<00:00, 12.83s/it]

Problem with nomao RF_2
Problem with nomao SVC_0
Problem with nomao XGB_2


In [7]:
results_all.to_csv(f"../results_anchor_all/anchor_mi_corr_all_local.csv", index=False)